# Введение в методы сбора и обработки данных
## Добашнее задание к уроку 2

In [24]:
import requests
import pprint
from collections import Counter
import re
import os
from shutil import rmtree

### Задание 1
Доработать приложение по поиску авиабилетов, чтобы оно возвращало билеты по названию города, а не по IATA коду. (У aviasales есть для этого дополнительное API) Пункт отправления и пункт назначения должны передаваться в качестве параметров. Сделать форматированный вывод, который содержит в себе пункт отправления, пункт назначения, дату вылета, цену билета (можно добавить еще другие параметры по желанию)

#### Решение:

Реализуем функцию которая переводит названия городов в их IATA

In [2]:
link = "https://www.travelpayouts.com/widgets_suggest_params"
string = f'Из {"архангельск"} в {"Красноярск"}'
req = requests.get(link, params={'q': string})
data = req.json()

In [3]:
def get_city_iata_by_name(name1: str, name2: str):
    """Возвращает IATA городов, принимая на вход их названия"""
    
    link = "https://www.travelpayouts.com/widgets_suggest_params"
    string = f'Из {name1} в {name2}'
    
    req = requests.get(link, params={'q': string})
    data = req.json()
    
    return data['origin']['iata'], data['destination']['iata']

# Проверим что функия отрабатыает корректно
get_city_iata_by_name('архангельск', 'Красноярск')

('ARH', 'KJA')

Реализуем функцию которая выдает список цен за билет в зависимости от указанных городов отправления и назначения:

In [4]:
def get_min_price(city1: str, city2: str, depart_date: str):
    link = "http://min-prices.aviasales.ru/calendar_preload"
    fromCity, toCity = get_city_iata_by_name(city1, city2)
    params = {
        'origin': fromCity,
        'destination': toCity,
        'depart_date': depart_date
    }
    
    req = requests.get(link, params=params)
    data = req.json()
    
    print(f'Результат поиска авиабилетов из {city1.capitalize()} в {city2.capitalize()} на {depart_date}:')
    
    if len(data['current_depart_date_prices']) == 0:
        print('Нет билетов подходящих запросам')
    else:      
        counter = 1
        for i in data['current_depart_date_prices']:
            print(f"{counter:2}) Цена: {i['value']:.0f}руб., билетная компания {i['gate']}")
            counter += 1 
              
        

In [5]:
get_min_price('Москва', 'Красноярск', '2019-09-19')

Результат поиска авиабилетов из Москва в Красноярск на 2019-09-19:
 1) Цена: 15070руб., билетная компания AMADEUS
 2) Цена: 15426руб., билетная компания AMADEUS
 3) Цена: 14926руб., билетная компания AMADEUS
 4) Цена: 10200руб., билетная компания Pobeda
 5) Цена: 23422руб., билетная компания MEGO.travel
 6) Цена: 13599руб., билетная компания Biletix
 7) Цена: 12957руб., билетная компания Aviakassa
 8) Цена: 13400руб., билетная компания Pobeda
 9) Цена: 11200руб., билетная компания Pobeda
10) Цена: 11200руб., билетная компания Pobeda
11) Цена: 11424руб., билетная компания KupiBilet.ru
12) Цена: 11200руб., билетная компания Pobeda
13) Цена: 15426руб., билетная компания AMADEUS
14) Цена: 12300руб., билетная компания Pobeda
15) Цена: 10700руб., билетная компания Pobeda
16) Цена: 10700руб., билетная компания Pobeda


Обернем запрос к пользователю в функцию

In [6]:
def prompt_user():
    """Запрашивает данные у пользователя и возвращает результат запроса к aviasales"""
    
    city1 = input("Укажите пункт отправления: ")
    city2 = input("Укажите пункт назначения:  ")
    depart_date = input("Дата вылета [гггг-мм-дд]: ")
    print()
    get_min_price(city1.capitalize(), city2.capitalize(), depart_date)

Наконец, попросим пользователя ввести города для поиска билетов:

In [7]:
prompt_user()

Укажите пункт отправления: Москва
Укажите пункт назначения:  Архангельск
Дата вылета [гггг-мм-дд]: 2019-10-01

Результат поиска авиабилетов из Москва в Архангельск на 2019-10-01:
 1) Цена: 10852руб., билетная компания AMADEUS
 2) Цена: 10852руб., билетная компания AMADEUS
 3) Цена: 10852руб., билетная компания AMADEUS
 4) Цена: 10852руб., билетная компания AMADEUS
 5) Цена: 10852руб., билетная компания AMADEUS
 6) Цена: 7900руб., билетная компания Nordavia
 7) Цена: 10852руб., билетная компания AMADEUS
 8) Цена: 10852руб., билетная компания AMADEUS
 9) Цена: 7900руб., билетная компания Nordavia
10) Цена: 10852руб., билетная компания AMADEUS
11) Цена: 10852руб., билетная компания AMADEUS
12) Цена: 10852руб., билетная компания AMADEUS
13) Цена: 10852руб., билетная компания AMADEUS
14) Цена: 10852руб., билетная компания AMADEUS


In [8]:
prompt_user()

Укажите пункт отправления: Мурманск
Укажите пункт назначения:  Красноярск
Дата вылета [гггг-мм-дд]: 2019-11-02

Результат поиска авиабилетов из Мурманск в Красноярск на 2019-11-02:
Нет билетов подходящих запросам


#### Заключение
В код можно было добавить ряд проверок, например:
1. Проверка соединения
2. Проверка наличия аэропорта в городе
3. Проверка наличия самого города
4. Защита от дурака при вводе данных

Но по сути это не меняет логики кода, а лишь усложняет код. Думаю, что для показательности работы с API его достаточно.

### Задание 2
В приложении парсинга википедии получить первую ссылку на другую страницу и вывести все значимые слова из неё. Результат записать в файл в форматированном виде

#### Решение:

In [12]:
def return_wiki_html(topic):
    """Возвращает html-код страницы"""
    wiki_request = requests.get(f'https://ru.wikipedia.org/wiki/{topic.capitalize()}')
    return wiki_request.text


def return_words(topic):
    """Возвращает 10 самых употребляемых на странице слов"""
    wiki_html = return_wiki_html(topic)
    words = re.findall('[а-яА-Я]{3,}', wiki_html)
    words_counter = Counter(words)
    
    return words_counter.most_common(10)


def find_first_link(topic):
    """Выводит первую ссылку со страницы"""
    text = return_wiki_html(topic)
    
    # Ссылки могут быть как внешние (http|https)  так и внутренние (/wiki)
    ans = re.findall("href\s*=\"((http|/wiki)\S*)\"", text)[1]
    if ans[1] == '/wiki':
        return f"https://ru.wikipedia.org{ans[0]}"
    else:
        return ans[0]
    

def write_stat_to_file(page_name, filename):
    """Записывает результат в файл"""
    link_most_common_words = return_words(page_name)
    with open(filename, 'w') as f:
        f.write(f'Список наиболее часто употребляемых слов на странице {first_link}:\n')
        f.writelines([f'{l[0]} - {l[1]} раз.\n' for l in link_most_common_words])

Пройдем по первой ссылке на искомой странице, и выведем оттуда в файл 10 наиболее часто используемых слов:

In [13]:
page_name = 'Государство'

# Найдем первую ссылку с данной странице
first_link = find_first_link(page_name)

In [14]:
write_stat_to_file(first_link, 'first_link_most_common_words')

In [15]:
!cat first_link_most_common_words

Список наиболее часто употребляемых слов на странице https://ru.wikipedia.org/wiki/%D0%A4%D0%B0%D0%B9%D0%BB:Leviathan_by_Thomas_Hobbes.jpg:
Файл - 11 раз.
Википедия - 6 раз.
Википедии - 6 раз.
Перейти - 6 раз.
страницы - 6 раз.
Поиск - 5 раз.
страницу - 5 раз.
Список - 5 раз.
для - 4 раз.
можно - 4 раз.


### Задание 3
Научить приложение определять количество ссылок в статье. Спарсить каждую ссылку и результаты записать в отдельные файлы.

#### Решение
Переопределим функцию `find_first_link` чтобы она возвращала весь список ссылок

In [16]:
def find_page_links(topic):
    """Выводит первую ссылку со страницы"""
    text = return_wiki_html(topic)
    
    # Ссылки могут быть как внешние (http|https)  так и внутренние (/wiki)
    ans = re.findall("href\s*=\"((http|/wiki)\S*)\"", text)
    
    result = []
    
    # Первая ссылка - ссылка страницы на саму себя. Так что ее опускаем
    for i in range(1, len(ans)):
        if ans[i][1] == '/wiki':
            result.append(f"https://ru.wikipedia.org{ans[i][0]}")
        else:
            result.append(ans[i][0])
    return result

In [17]:
page_name = 'Теория вероятностей'
page_links = find_page_links(page_name)

Создадим директорию куда будем складывать файлы

In [25]:
links_dirname = 'links_parse_info'
if os.path.isdir(links_dirname):
    rmtree(links_dirname, ignore_errors=True)
    
os.mkdir(links_dirname)
os.chdir(links_dirname)

In [27]:
counter = 1

for link in page_links:
    write_stat_to_file(link, f'link_{counter}')
    counter += 1

Подсчитаем количество получившихся файлов

In [28]:
len(os.listdir())

287